In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score,accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from aif360.metrics import ClassificationMetric

c:\Users\Diego\anaconda3\Lib\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
c:\Users\Diego\anaconda3\Lib\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_disc

Load and Pre-processing Data Function


In [3]:
def process_and_load_data(data, target = str):
    #The input should only be a Pandas DataFrame 
    try:
        (data == pd.DataFrame)
        X = data.drop(target, axis = 1)
        y = data[target]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        return X_train, X_test, X_val, y_train, y_test, y_val
    except:
        print("Input needs to be a Pandas DataFrame.")

 


# Model Accuracy & Fairness Assessment Functions

## Function to test the Model's Accuracy

In [ ]:
def model_assessment(input, predictions, actuals):
    male_indices = input[input['Gender'] == 1].index.tolist()
    female_indices = input[input['Gender'] == 2].index.tolist()

    #Overall
    overall_accuracy = accuracy_score(actuals, predictions)
    overall_recall = recall_score(actuals, predictions)
    overall_precision = precision_score(actuals, predictions)

    #Female
    female_preds = predictions[female_indices]
    female_actuals = actuals[female_indices]
    female_accuracy = accuracy_score(female_actuals, female_preds)
    female_recall = recall_score(female_actuals, female_preds)
    female_precision = precision_score(female_actuals, female_preds)

    #Male
    male_preds = predictions[male_indices]
    male_actuals = actuals[male_indices]
    male_accuracy = accuracy_score(male_actuals, male_preds)
    male_recall = recall_score(male_actuals, male_preds)
    male_precision = precision_score(male_actuals, male_preds)

    results = pd.DataFrame({
        'Group': ['Overall', 'Female', 'Male'],
        'Accuracy': [overall_accuracy, female_accuracy, male_accuracy],
        'Recall': [overall_recall, female_recall, male_recall],
        'Precision': [overall_precision, female_precision, male_precision]
    })

    return results





## Function to test the Model's Fairness between Males and Females